## Exercise Set 1

### Multiple Choice 
**Q1.** Which of these are NOT key components for an optimization model?
- decision variables
- parameters
- objective function
- hyperparameters

**Q2.** MO can be used as a strict replacement for nearly all ML problems
- True
- False

**Q3.** ML can be used as a strict replacement for nearly all MO problems
- True
- False

**Q4.** Which of the following is a type of decision variables in mathematical optimization?
- binary
- complex
- feature
- complimentary

**Q5.** To solve a mixed-integer program, a solver may need to solve this many linear programs:
- none
- the number of decision variables
- the number of total constraints
- possibly exponentially many

**Q6.** It is required that you or your data analytics team develop an expertise in `predictive` analytics before attempting `prescriptive` analytics (like mathematical optimization).
- True
- False

**Q7.** The `feasible region` of a linear program (LP) will have ____ points in it than its corresponding mixed-integer program (MIP), assuming the two models are exactly the same other than the LP has only continuous variables and the MIP contains integer variables.
- more
- less
- always exactly the same

**Q8.** Unless specified, the default variable type when using `addVars()` is **continuous**.
- True
- False

Let $J = \{\texttt{Apple, Banana, Coconut, Dragonfruit, Elderberry, Fig, Gooseberry}\}$ and $T = \{1, 2, 3, 4\}$ 

**Q9-a.** Adding decision variables using `addVars(J,...)` and `addVars(range(6),...)` will add the *same* number of variables to a model
- True
- False

**Q9-b.** Using the sets above, adding decision variables using `addVars(J, T,...)` and `addVars(range(28),...)` will add the *same* number of variables to a model
- True
- False

### Formulation and Coding 
Below is code for the entire original model in one cell if you would like to use it to help with these exercises.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 

m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship')
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 
m.optimize()

You are told there is a new policy for transporting **widgets** from production facilities. It is now required that the minimum number of widgets shipped from any production facility to any distribution center needs to be at least 20. 

**Q10-a.** Write out how the formulation changes using mathematical notation given the new requirement.

**Q10-b.** Write the changes for **Q11-a** in gurobipy code in the cell below (no need to run it, unless you want to copy the model here to check).

The initial widget model $m$ represented a single time period (e.g. a week, month, quarter). Suppose we added a time component using a set $T = \{0, 1, 2, 3\}$ representing a quarter of a year. 
**Q11-a.** Use `addVar()` or `addVars()` to create a decision variable in gurobipy that represents the number of widgets shipped from a production facility to a distribution center for a given time period. 

**Q11-b.** To reference a time period before a given time $t$, we can use $t-1$ as a subscript (since $T$ is a set of integers), but $t-1$ doesn't work when $t = 0$ since $-1$ isn't in $T$. Fill in the ??? in the code below to represent a set of constraints that limits the amount a production facility can produce to `max_prod[p]` over *two consecutive* time periods. 

In [ ]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,???] + x[p,d,???] for d in distribution) <= max_prod[p] for p in production for t in ??? if t ???), name = 'time_prod_limit')

**Q12.** Rewrite the `meet_demand` constraint using one or two `for` loops.

In [16]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 

m = gp.Model('widgets')
x = m.addVars(production, distribution, range(4), name = 'prod_ship')
m.update()
x

{('Baltimore', 'Columbia', 0): <gurobi.Var prod_ship[Baltimore,Columbia,0]>,
 ('Baltimore', 'Columbia', 1): <gurobi.Var prod_ship[Baltimore,Columbia,1]>,
 ('Baltimore', 'Columbia', 2): <gurobi.Var prod_ship[Baltimore,Columbia,2]>,
 ('Baltimore', 'Columbia', 3): <gurobi.Var prod_ship[Baltimore,Columbia,3]>,
 ('Baltimore',
  'Indianapolis',
  0): <gurobi.Var prod_ship[Baltimore,Indianapolis,0]>,
 ('Baltimore',
  'Indianapolis',
  1): <gurobi.Var prod_ship[Baltimore,Indianapolis,1]>,
 ('Baltimore',
  'Indianapolis',
  2): <gurobi.Var prod_ship[Baltimore,Indianapolis,2]>,
 ('Baltimore',
  'Indianapolis',
  3): <gurobi.Var prod_ship[Baltimore,Indianapolis,3]>,
 ('Baltimore', 'Lexington', 0): <gurobi.Var prod_ship[Baltimore,Lexington,0]>,
 ('Baltimore', 'Lexington', 1): <gurobi.Var prod_ship[Baltimore,Lexington,1]>,
 ('Baltimore', 'Lexington', 2): <gurobi.Var prod_ship[Baltimore,Lexington,2]>,
 ('Baltimore', 'Lexington', 3): <gurobi.Var prod_ship[Baltimore,Lexington,3]>,
 ('Baltimore', 'Nash

In [17]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,t] + x[p,d,t-1] for d in distribution) <= max_prod[p] for p in production for t in range(4) if t > 0), name = 'time_prod_limit')
m.update()
time_prod_limit

{('Baltimore', 1): <gurobi.Constr time_prod_limit[Baltimore,1]>,
 ('Baltimore', 2): <gurobi.Constr time_prod_limit[Baltimore,2]>,
 ('Baltimore', 3): <gurobi.Constr time_prod_limit[Baltimore,3]>,
 ('Cleveland', 1): <gurobi.Constr time_prod_limit[Cleveland,1]>,
 ('Cleveland', 2): <gurobi.Constr time_prod_limit[Cleveland,2]>,
 ('Cleveland', 3): <gurobi.Constr time_prod_limit[Cleveland,3]>,
 ('Little Rock', 1): <gurobi.Constr time_prod_limit[Little Rock,1]>,
 ('Little Rock', 2): <gurobi.Constr time_prod_limit[Little Rock,2]>,
 ('Little Rock', 3): <gurobi.Constr time_prod_limit[Little Rock,3]>,
 ('Birmingham', 1): <gurobi.Constr time_prod_limit[Birmingham,1]>,
 ('Birmingham', 2): <gurobi.Constr time_prod_limit[Birmingham,2]>,
 ('Birmingham', 3): <gurobi.Constr time_prod_limit[Birmingham,3]>,
 ('Charleston', 1): <gurobi.Constr time_prod_limit[Charleston,1]>,
 ('Charleston', 2): <gurobi.Constr time_prod_limit[Charleston,2]>,
 ('Charleston', 3): <gurobi.Constr time_prod_limit[Charleston,3]>}